In [6]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from sklearn.datasets import load_boston
from sklearn.preprocessing import StandardScaler
from torchinfo import summary
import numpy as np

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


In [7]:
# Om			\Omega_m
# Obh2			\Omega_{b}h^2
# h	
datafile = 'chains/LCDM_phy_HD_nested_dynesty_multi_1.txt'

In [4]:
class LoadDataSet(torch.utils.data.Dataset):
    '''
    Prepare the dataset for regression
    '''
    def __init__(self, X, y, scale_data=True):
        if not torch.is_tensor(X) and not torch.is_tensor(y):
            # Apply scaling if necessary
            if scale_data:
                X = StandardScaler().fit_transform(X)
            self.X = torch.from_numpy(X)
            self.y = torch.from_numpy(y)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, i):
        return self.X[i], self.y[i]

In [30]:
class MLP(nn.Module):
    '''
    Multilayer Perceptron for regression.
    '''
    def __init__(self):
        super().__init__()
        ncols = 3
        self.layers = nn.Sequential(
          nn.Linear(ncols, 64),
          nn.ReLU(),
          nn.Linear(64, 32),
          nn.ReLU(),
          nn.Linear(32, 1)
        )


    def forward(self, x):
        '''
          Forward pass
        '''
        return self.layers(x)

In [31]:
if __name__ == '__main__':
    # Set fixed random number seed
    torch.manual_seed(42)
  
    # Load Boston dataset
    X = np.loadtxt(datafile, usecols=(2,3,4))
    y = np.loadtxt(datafile, usecols=1).reshape(-1, 1)
    print(np.shape(X), np.shape(y))

(1036, 3) (1036, 1)


In [32]:
dataset = LoadDataSet(X, y)

In [33]:
dataset

In [34]:
trainloader = torch.utils.data.DataLoader(dataset, batch_size=10, shuffle=True, num_workers=1)

In [35]:
dataset.y

tensor([[113.1624],
        [105.5863],
        [102.7997],
        ...,
        [  7.2537],
        [  7.2515],
        [  7.2512]], dtype=torch.float64)

In [36]:
# Initialize the MLP
mlp = MLP()
mlp.float()
# Define the loss function and optimizer
loss_function = nn.L1Loss()
optimizer = torch.optim.Adam(mlp.parameters(), lr=1e-4)

In [37]:
summary(mlp, batch_size=-1)

Layer (type:depth-idx)                   Param #
MLP                                      --
├─Sequential: 1-1                        --
│    └─Linear: 2-1                       256
│    └─ReLU: 2-2                         --
│    └─Linear: 2-3                       2,080
│    └─ReLU: 2-4                         --
│    └─Linear: 2-5                       33
Total params: 2,369
Trainable params: 2,369
Non-trainable params: 0

In [39]:
# Run the training loop
for epoch in range(0, 5): # 5 epochs at maximum  
    # Print epoch
    print(f'Starting epoch {epoch+1}')

    # Set current loss value
    current_loss = 0.0

    # Iterate over the DataLoader for training data
    for i, data in enumerate(trainloader, 0):

          # Get and prepare inputs
        inputs, targets = data
        inputs, targets = inputs.float(), targets.float()
        targets = targets.reshape((targets.shape[0], 1))

        # Zero the gradients
        optimizer.zero_grad()

        # Perform forward pass
        outputs = mlp(inputs)

        # Compute loss
        loss = loss_function(outputs, targets)

        # Perform backward pass
        loss.backward()

        # Perform optimization
        optimizer.step()

        # Print statistics
        current_loss += loss.item()
        if i % 10 == 0:
            print('Loss after mini-batch %5d: %.3f' %
                (i + 1, current_loss / 500))
            current_loss = 0.0

# Process is complete.
print('Training process has finished.')

Starting epoch 1
Loss after mini-batch     1: 0.014
Loss after mini-batch    11: 0.207
Loss after mini-batch    21: 0.233
Loss after mini-batch    31: 0.187
Loss after mini-batch    41: 0.214
Loss after mini-batch    51: 0.156
Loss after mini-batch    61: 0.208
Loss after mini-batch    71: 0.168
Loss after mini-batch    81: 0.163
Loss after mini-batch    91: 0.157
Loss after mini-batch   101: 0.209
Starting epoch 2
Loss after mini-batch     1: 0.023
Loss after mini-batch    11: 0.160
Loss after mini-batch    21: 0.138
Loss after mini-batch    31: 0.190
Loss after mini-batch    41: 0.142
Loss after mini-batch    51: 0.142
Loss after mini-batch    61: 0.157
Loss after mini-batch    71: 0.159
Loss after mini-batch    81: 0.152
Loss after mini-batch    91: 0.200
Loss after mini-batch   101: 0.112
Starting epoch 3
Loss after mini-batch     1: 0.008
Loss after mini-batch    11: 0.136
Loss after mini-batch    21: 0.163
Loss after mini-batch    31: 0.110
Loss after mini-batch    41: 0.110
Loss

In [40]:
mlp.eval()

MLP(
  (layers): Sequential(
    (0): Linear(in_features=3, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=32, bias=True)
    (3): ReLU()
    (4): Linear(in_features=32, out_features=1, bias=True)
  )
)

In [41]:
# test = np.random.randn(13)
# torch usa tensores de torch y no numpy.darrays
dtype = torch.float
test = torch.randn((1, 3), device=device, dtype=dtype)

In [42]:
mlp.forward(test)

tensor([[8.8453]], grad_fn=<AddmmBackward0>)